In [226]:
import magpylib as magpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import einops as eo

import sklearn
from scipy.interpolate import RBFInterpolator
from scipy.interpolate import LinearNDInterpolator 
 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline


In [ ]:
class DiscreteSensor(magpy.Sensor):
    def __init__(self, sensitivity=None, noise=None, position=(0,0,0), pixel=(0,0,0), orientation=None, style=None, **kwargs):
        self.sensitivty=sensitivity
        self.noise=noise
        super().__init__(position=position, pixel=pixel, orientation=orientation, style=style)
    def getB(self, *observers, squeeze=True, pixel_agg=None, output='ndarray'):
        B = getB(self, *observers, squeeze=squeeze, pixel_agg=pixel_agg, output=output)
        if self.noise!=None:
            B = B+(2*np.random.rand(*B.shape)-1)*self.noise
        if self.sensitivity!=None:
            B = round(B/self.sensitivity)*self.sensitivity

In [227]:
Br_max = 1210 #mT
d = (3/16)*25.4
h = (1/8)*25.4

In [228]:
M = Br_max


In [229]:
magnet = magpy.magnet.Cylinder(magnetization = (0, 0, M), dimension=(d, h), position=(0, 0, 33))
sensor = DiscreteSensor(sensitivity=(1/6842)/10, noise=4.1e-3)
B = magnet.getB(sensor)
print(B)


[0.         0.         0.30211551]


In [230]:
step_xy = 0.2
bounds_xy = [-7,7]
samples_xy =int(((bounds_xy[1]-bounds_xy[0]))/step_xy) + 1


bounds_z = [0, 7]
step_z = 0.2
samples_z = int((bounds_z[1]-bounds_z[0])/step_z)+1

xx, step_xx = np.linspace(*bounds_xy, num=samples_xy, retstep=True)
yy, step_yy = np.linspace(*bounds_xy, num=samples_xy, retstep=True)
zz, step_zz = np.linspace(*bounds_z, num=samples_z, retstep=True)

grid = eo.rearrange(np.array(np.meshgrid(xx,yy,zz)), "d x y z -> (x y z) d")

magnet.move(grid, start=0)

B_cal = sensor.getB(magnet)


In [231]:
interpolator = LinearNDInterpolator(grid, B_cal)

In [235]:
step_xy = 0.03
bounds_xy = [-3,3]
samples_xy =int(((bounds_xy[1]-bounds_xy[0]))/step_xy) + 1


bounds_z = [2, 5]
step_z = 0.03
samples_z = int((bounds_z[1]-bounds_z[0])/step_z)+1

xx, step_xx = np.linspace(*bounds_xy, num=samples_xy, retstep=True)
yy, step_yy = np.linspace(*bounds_xy, num=samples_xy, retstep=True)
zz, step_zz = np.linspace(*bounds_z, num=samples_z, retstep=True)

grid = eo.rearrange(np.array(np.meshgrid(xx,yy,zz)), "d x y z -> (x y z) d")
magnet.reset_path()
magnet.position = (0, 0, 33)
magnet.move(grid, start=0)

B_meas = sensor.getB(magnet)


In [236]:
X_fit = interpolator(B_meas)

In [ ]:
((X_fit-grid)**2).mean()